In [17]:
# test marker API
import requests
import time
import os
import json
import tqdm.notebook as tqdm
headers = {"X-Api-Key": "hArnNmJTVNiaGr45NJAqj3Xq1IOeX0ge6des-bWgeRU"}
url = "https://www.datalab.to/api/v1/marker"

In [18]:
pdf_dir = "/Users/yiquntchen/Downloads/guidelines_downloads/"
output_dir = "/Users/yiquntchen/Downloads/guidelines_downloads_markdown/"
os.makedirs(output_dir, exist_ok=True)

for file_id in tqdm.tqdm(sorted(os.listdir(pdf_dir)), desc="Processing PDFs"):
    try:
        # 1. Prepare the form data
        pdf_path = os.path.join(pdf_dir, file_id)
        output_path = os.path.join(output_dir, f"{file_id.split('pdf')[0]}json")

        # Skip if JSON already exists
        if os.path.exists(output_path):
            tqdm.tqdm.write(f"Skipping file {file_id}: JSON already exists.")
            continue        

        if not os.path.exists(pdf_path):
            tqdm.tqdm.write(f"Warning: PDF not found for file {file_id}, skipping.")
            continue

        with open(pdf_path, "rb") as f:
            form_data = {
                'file': (f'{file_id}', f, 'application/pdf'),
                'langs': (None, "English"),
                "force_ocr": (None, True),
                "paginate": (None, False),
                'output_format': (None, 'markdown'),
                "use_llm": (None, False),
                "strip_existing_ocr": (None, True),
                "disable_image_extraction": (None, False)
            }

            # 2. Send the POST request
            response = requests.post(url, files=form_data, headers=headers)
            data = response.json()

        # 3. Get the check URL and start polling
        check_url = data["request_check_url"]
        max_polls = 100
        success = False

        for _ in range(max_polls):
            time.sleep(2)
            poll_response = requests.get(check_url, headers=headers)
            poll_data = poll_response.json()

            if poll_data["status"] == "complete":
                success = True
                break

        # 4. Save the result if successful
        if success:
            with open(output_path, "w") as f_out:
                json.dump(poll_data, f_out)
            tqdm.tqdm.write(f"Saved file {file_id} result.")
        else:
            tqdm.tqdm.write(f"Warning: file {file_id} did not complete in time.")

    except Exception as e:
        tqdm.tqdm.write(f"Error processing file {file_id}: {e}")

Processing PDFs:   0%|          | 0/290 [00:00<?, ?it/s]

Skipping file 2023-11-15-ishlt-ast-asts-joint-statement-covid19-vaccination.pdf: JSON already exists.
Skipping file 20250605_Board_of_Governors_Agenda.pdf: JSON already exists.
Skipping file ACG-H.-pylori-Guideline-Summary.pdf: JSON already exists.
Skipping file AsthmaManagementGuidelinesReport-2-4-21.pdf: JSON already exists.
Skipping file CA_LA_1_Revision_18_DMHC_-_S_CHSO_tcm17-41485.pdf: JSON already exists.
Skipping file CA_QM_12_Revision_17_DMHC_-_S_tcm17-77421.pdf: JSON already exists.
Skipping file CA_QM_12_Revision_9_CDI_-_S_tcm17-91869.pdf: JSON already exists.
Skipping file CD-PPX-abx-algorithm.pdf: JSON already exists.
Skipping file CPG%20111%20Revision%202%20-%20S%20-%20Effective%20010123_tcm17-211622.pdf: JSON already exists.
Skipping file CPG_102_Revision_19_-_S_tcm17-41734.pdf: JSON already exists.
Skipping file CPG_103_Revision_18_-_S_tcm17-41735.pdf: JSON already exists.
Skipping file CPG_104_Revision_18_-_S_tcm17-41736.pdf: JSON already exists.
Skipping file CPG_106_R

In [19]:
output_dir

'/Users/yiquntchen/Downloads/guidelines_downloads_markdown/'

In [23]:
# Find all JSON files in the input directory
json_files = [f for f in os.listdir(output_dir) if f.endswith(".json")]
output_file = os.path.join("/Users/yiquntchen/Desktop/chen-lab/MEDAL/medical_guidelines_all_text.jsonl")

with open(output_file, "w") as outfile:
    for json_file in tqdm.tqdm(json_files, desc="Processing JSONs"):
        pmid = json_file.replace(".json", "")
        json_path = os.path.join(output_dir, json_file)

        try:
            with open(json_path, "r") as f:
                data = json.load(f)

            markdown_text = data.get("markdown")
            if markdown_text is not None:
                entry = {"text-guideline": pmid, "text": markdown_text}
                outfile.write(json.dumps(entry) + "\n")
            else:
                tqdm.write(f"Warning: No 'markdown' found in {json_file}, skipping.")
        
        except Exception as e:
            tqdm.write(f"Error processing {json_file}: {e}")

Processing JSONs:   0%|          | 0/289 [00:00<?, ?it/s]